In [4]:
import numpy as np
import ccxt
class Environment:
    def __init__(self, history_length=50):
        self.history_length = history_length
        self.reset()
        
        

    def reset(self):
        self.t = self.history_length
        self.profit = 1
        self.positions = []
        self.position_value = 0
        self.action_b = 0
        self.action_s = 0
        self.balance = 100000
        self.shares = 0
        self.history = [0 for _ in range(self.history_length)]
        #self.history = self.data.iloc[:self.history_length].to_numpy()
        # return the state/observation representation
        return self.history
    

    def __call__(self, price, macd, prev_macd):
        self.exchange = ccxt.binance({
            'verbose': False,
            'apiKey': "4BsKcSV5PKr04SvKBmdpqM4RXdNoScwRxFnKpFdNYQ5N1EtjIbHePjrolqlyGwBy",
            'secret': "Ivdb3W4KOmEhwIaggn9ommaAg9LF6MqanwNEIU0DQJlCM1NhwTbuq6c8lkldcDmZ",
            'options': { 'adjustForTimeDifference': True }})
        if macd*prev_macd<0 and macd>prev_macd:
            try:
                self.balance = self.exchange.fetch_balance()["free"]["BUSD"]
                if self.balance > 1:
                    self.exchange.create_order(
                        symbol='XRP/BUSD',
                        type='market',
                        side='buy',
                        #price=0.013,
                        amount=(self.balance/price),
                    )
            except Exception as e:
                print(e)
                #self.__call__( price, macd, prev_macd)
            print("==============Bought {} shares for {}===============".format(self.balance/price, price))
            
        elif macd*prev_macd<0 and macd<prev_macd:
            try:
                self.shares = self.exchange.fetch_balance()["free"]["XRP"]
                if self.shares > 0:
                    self.exchange.create_order(
                        symbol='XRP/BUSD',
                        type='market',
                        side='sell',
                        #price=0.013,
                        amount=self.shares,
                    )
            except Exception as e:
                print(e)
                #self.__call__(price, macd, prev_macd)
            print("================Sold {} shares for {}===============".format(self.shares, price))

In [5]:

from twelvedata import TDClient

apikey = "7b0d2ab7a04041f3999b57dcc508867c"
td = TDClient(apikey)
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from math import floor
import time
from tqdm import tqdm
import random
from objsize import get_deep_size
from random import randrange
import datetime
from datetime import datetime

def get_data(ticker, interval="5min", feed_window=10, size=30, whole_day=True):
    x, y = [], []
    for i in range(size):
        try:
            df = td.time_series(
                symbol=ticker,
                outputsize=feed_window,
                interval=interval,
                timezone='Europe/Moscow',
                start_date='2020-01-01 9:30',
                end_date=datetime.now()
            )

            df = df.with_macd(signal_period=40, fast_period=12, slow_period=26)

            df = df.as_pandas().drop(['high', 'low', 'open'], axis=1).iloc[::-1]
            #df = df.to_numpy()
            #df = np.array(df)
        except Exception as e:
            time.sleep(1)
            size -= 1
            print(e)
            continue
        x = df

    return x


x = get_data(ticker="XRP/USD", feed_window=2, size=1)
print(float(x["macd_hist"].iloc[0]),float(x["close"].iloc[0]))
print(x.index[1])
print(x)

-0.00421 1.4083
2021-04-27 15:45:00
                      close     macd  macd_signal  macd_hist
datetime                                                    
2021-04-27 15:40:00  1.4083 -0.00097      0.00323   -0.00421
2021-04-27 15:45:00  1.4130 -0.00158      0.00300   -0.00457


In [3]:
import time
def prod():
    env = Environment()
    x = get_data(ticker="XRP/USD", feed_window=2, size=1)
    time_ = x.index[1]
    while True:
        try:
            x = get_data(ticker="XRP/USD", feed_window=2, size=1)
            if time_ == x.index[1]:
                time.sleep(1)
                #print(time_)
                continue
            else:
                time_ = x.index[1]
                #print(time_,"----------------------")
                print("Time:",datetime.now().strftime("20%y-%m-%d %H:%M"),
                      " |  Price:","{:.3f}".format(float(x["close"].iloc[1])),
                      " |  MACD:", "{:.5f}".format(float(x["macd_hist"].iloc[1])))
                env(float(x["close"].iloc[1]), x["macd_hist"].iloc[1],x["macd_hist"].iloc[0])
        except Exception as e:
            print(e)

prod() 
    

Time: 2021-04-27 15:47  |  Price: 1.413  |  MACD: -0.00107
Time: 2021-04-27 15:48  |  Price: 1.415  |  MACD: -0.00065
Time: 2021-04-27 15:49  |  Price: 1.413  |  MACD: -0.00046


KeyboardInterrupt: 

In [61]:
import time
for _ in range(3):
    time.sleep(3)

In [90]:
exchange = ccxt.binance({
            'verbose': False,
            'apiKey': "4BsKcSV5PKr04SvKBmdpqM4RXdNoScwRxFnKpFdNYQ5N1EtjIbHePjrolqlyGwBy",
            'secret': "Ivdb3W4KOmEhwIaggn9ommaAg9LF6MqanwNEIU0DQJlCM1NhwTbuq6c8lkldcDmZ",
        })